<a href="https://colab.research.google.com/github/SJG0510/Private-Project-Practice/blob/main/Health.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

한국어 사용

In [ ]:
import os
import shutil

# matplotlib 폰트 캐시 경로 (코랩 환경)
cache_dirs = [
    os.path.expanduser('~/.cache/matplotlib'),
    '/root/.cache/matplotlib'
]

for cache_dir in cache_dirs:
    if os.path.exists(cache_dir):
        shutil.rmtree(cache_dir)
        print(f"Deleted font cache folder: {cache_dir}")
    else:
        print(f"Font cache folder not found: {cache_dir}")

import matplotlib.font_manager as fm

# Nanum Gothic 폰트 경로
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'

# 폰트 등록
fm.fontManager.addfont(font_path)

# 폰트 프로퍼티 생성
fontprop = fm.FontProperties(fname=font_path, size=12)

print("한글 폰트 설정 완료")

Font cache folder not found: /root/.cache/matplotlib
Font cache folder not found: /root/.cache/matplotlib
한글 폰트 설정 완료


데이터 로드

In [ ]:
import pandas as pd
import urllib.parse

# GitHub raw 파일 기본 경로
base_url = "https://raw.githubusercontent.com/SJG0510/Private-Project-Practice/data/"

file_names = {
    'smoking': '시도_현재흡연.csv',           # 현재 흡연율 데이터
    'monthly_drinking': '시도_월간음주.csv',  # 월간 음주율 데이터
    'highrisk_drinking': '시도_고위험음주.csv', # 고위험 음주율 데이터
    'obesity': '시도_비만율.csv',             # 비만율 데이터
    'depression': '시도_우울감.csv',           # 우울감 경험률 데이터
    'depressive_symptom': '시도_우울증상.csv',  # 우울증상 경험률 데이터
    'stress': '시도_스트레스.csv',             # 스트레스 인지율 데이터
}

def load_csv_files(base_url, file_dict):
    data = {}
    for key, filename in file_dict.items():
        encoded_filename = urllib.parse.quote(filename)
        url = base_url + encoded_filename
        try:
            try:
                df = pd.read_csv(url, encoding='utf-8-sig')
            except UnicodeDecodeError:
                df = pd.read_csv(url, encoding='cp949')
            data[key] = df
            print(f"{key} loaded successfully")
        except Exception as e:
            error_msg = str(e).encode('ascii', errors='ignore').decode()
            print(f"{key} load failed: {error_msg}")
    return data

data_dict = load_csv_files(base_url, file_names)

# 데이터 일부 확인
for key, df in data_dict.items():
    print(f"\n[{key}] sample data:")
    print(df.head(5))

smoking loaded successfully
monthly_drinking loaded successfully
highrisk_drinking loaded successfully
obesity loaded successfully
depression loaded successfully
depressive_symptom loaded successfully
stress loaded successfully

[smoking] sample data:
   구분 Unnamed: 1 Unnamed: 2       2008       2009       2010       2011  \
0  서울          N        (명)     21,913     23,175     22,897     23,115   
1  서울         전체       (조율)  24.2(0.3)  24.3(0.3)  23.2(0.3)  22.3(0.3)   
2  서울        NaN     (표준화율)  24.6(0.3)  24.7(0.3)  23.7(0.3)  23.0(0.3)   
3  서울          남        NaN  45.6(0.6)  45.5(0.5)  43.5(0.5)  42.0(0.5)   
4  서울          여        NaN   3.7(0.2)   4.0(0.2)   3.6(0.2)   3.5(0.2)   

        2012       2013       2014  ...       2016       2017       2018  \
0     23,059     23,136     23,026  ...     22,950     22,955     22,904   
1  22.2(0.3)  21.7(0.3)  20.6(0.3)  ...  19.5(0.3)  18.8(0.3)  18.6(0.3)   
2  23.0(0.3)  22.6(0.3)  21.4(0.3)  ...  20.5(0.3)  20.0(0.3)  19.8(0

전처리

In [ ]:
import re

def preprocess_smoking_data(df):
    # '95% CI' 컬럼 제거
    if '95% CI' in df.columns:
        df = df.drop(columns=['95% CI'])

    # 'Unnamed: 1' 컬럼 기준으로 '전체', '남', '여', 연령대 필터링
    전체_df = df[df['Unnamed: 1'] == '전체'].copy()
    성별_df = df[df['Unnamed: 1'].isin(['남', '여'])].copy()
    연령_df = df[df['Unnamed: 1'].isin(['19-29', '30-39', '40-49', '50-59', '60-69', '70이상'])].copy()

    # 숫자 데이터에서 괄호, 단위 제거 및 숫자형 변환 함수
    def to_numeric(val):
        if pd.isna(val):
            return None
        s = re.findall(r"[-+]?\d*\.\d+|\d+", str(val))
        return float(s[0]) if s else None

    # 연도 컬럼만 선택 (2008~2024)
    year_cols = [col for col in df.columns if re.match(r"^\d{4}$", str(col))]

    # 각 데이터프레임 숫자형 변환
    for year in year_cols:
        전체_df[year] = 전체_df[year].apply(to_numeric)
        성별_df[year] = 성별_df[year].apply(to_numeric)
        연령_df[year] = 연령_df[year].apply(to_numeric)

    # 인덱스 초기화
    전체_df.reset_index(drop=True, inplace=True)
    성별_df.reset_index(drop=True, inplace=True)
    연령_df.reset_index(drop=True, inplace=True)

    return 전체_df, 성별_df, 연령_df

# 불러온 데이터 전처리
smoking_df = data_dict['smoking']
전체, 성별, 연령 = preprocess_smoking_data(smoking_df)

print("전체 데이터 샘플")
print(전체.head(5))
print("\n성별 데이터 샘플")
print(성별.head(5))
print("\n연령 데이터 샘플")
print(연령.head(5))

전체 데이터 샘플
   구분 Unnamed: 1 Unnamed: 2  2008  2009  2010  2011  2012  2013  2014  2015  \
0  서울         전체       (조율)  24.2  24.3  23.2  22.3  22.2  21.7  20.6  19.4   
1  부산         전체       (조율)  26.1  26.5  24.6  24.3  23.5  23.1  22.3  20.6   
2  대구         전체       (조율)  25.0  25.6  24.5  24.4  24.5  23.7  22.9  20.4   
3  인천         전체       (조율)  26.5  26.9  27.1  25.9  25.8  24.3  25.4  23.8   
4  광주         전체       (조율)  21.2  22.1  22.2  22.7  23.0  21.9  22.2  18.6   

   2016  2017  2018  2019  2020  2021  2022  2023  2024  
0  19.5  18.8  18.6  16.7  15.5  14.9  15.3  16.6  14.8  
1  20.5  20.4  19.6  17.7  17.4  16.6  17.1  17.2  15.9  
2  19.6  19.6  20.5  18.5  18.3  17.8  17.0  18.2  17.9  
3  24.1  22.2  22.3  19.9  19.8  19.4  19.4  18.7  18.3  
4  18.8  20.1  19.9  18.6  17.2  16.3  17.1  16.9  15.6  

성별 데이터 샘플
   구분 Unnamed: 1 Unnamed: 2  2008  2009  2010  2011  2012  2013  2014  2015  \
0  서울          남        NaN  45.6  45.5  43.5  42.0  41.6  40.6  38.5  36.2  

학습 및 예측

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

def train_linear_regression(X, y):
    """
    선형회귀 모델 학습
    X : 2차원 numpy array, 입력 (연도)
    y : 1차원 numpy array, 목표값 (흡연율)
    """
    model = LinearRegression()
    model.fit(X, y)
    return model

def fit_region_category(df, region, category):
    """
    특정 지역(region), 특정 구분(category)의 흡연율 데이터로 선형회귀 모델 학습
    df : 전체, 성별, 또는 연령별 DataFrame 중 하나
    region : 예) '서울'
    category : 예) '전체', '남', '여', '19-29' 등 (df의 'Unnamed: 1' 컬럼 기준)
    """
    sub_df = df[(df['구분'] == region) & (df['Unnamed: 1'] == category)]
    if sub_df.empty:
        print(f"{region} - {category} 데이터 없음")
        return None, None, None

    # 연도 컬럼명 리스트 (문자열)
    year_cols = [col for col in df.columns if col.isdigit()]
    # X: 연도 숫자, 2차원 배열 (n_samples, 1)
    X = np.array([int(year) for year in year_cols]).reshape(-1, 1)
    # y: 해당 지역-구분의 연도별 흡연율 값, 1차원 배열
    y = sub_df[year_cols].values.flatten().astype(float)

    model = train_linear_regression(X, y)
    return model, X.flatten(), y


In [ ]:
def show_prediction(df, region, category, predict_years=2):
    model, actual_years, actual_rates = fit_region_category(df, region, category)
    if model is None:
        print(f"[{region} - {category}] 데이터가 없어 예측을 건너뜁니다.")
        return

    last_year = actual_years[-1]
    pred_years, pred_rates = predict_future(model, last_year, predict_years)

    # 실제 데이터 출력
    print(f"\n[{region} - {category}] 학습 데이터 연도:")
    print(actual_years)
    print(f"[{region} - {category}] 실제 흡연율:")
    print(actual_rates)

    # 예측 데이터 출력
    print(f"[{region} - {category}] 예측 연도:")
    print(pred_years)
    print(f"[{region} - {category}] 예측 흡연율:")
    print(pred_rates)

    # 시각화
    plt.figure(figsize=(10,6))
    plt.plot(actual_years, actual_rates, marker='o', label="실제 흡연율")
    plt.plot(pred_years, pred_rates, 'r--', marker='x', label="예측 흡연율")
    plt.xlabel("연도", fontproperties=fontprop)
    plt.ylabel("흡연율 (%)", fontproperties=fontprop)
    plt.title(f"{region} - {category} 흡연율 예측", fontproperties=fontprop)
    plt.legend(prop=fontprop)
    plt.grid(True)
    plt.xticks(list(range(min(actual_years), max(pred_years)+1)))
    plt.show()

def interactive_prediction(df, predict_years=2):
    print("예측 가능한 지역명 목록:")
    print(", ".join(df['구분'].unique()))
    print("\n예측 가능한 구분명 목록:")
    print("전체, 남, 여, 19-29, 30-39, 40-49, 50-59, 60-69, 70이상")

    region = input("예측할 지역명을 입력하세요: ").strip()
    categories_input = input("예측할 구분명을 쉼표로 구분하여 입력하세요: ")
    categories = [c.strip() for c in categories_input.split(',')]

    for category in categories:
        show_prediction(df, region, category, predict_years)

# 실행: 직접 입력으로 동작
interactive_prediction(전체, predict_years=2)

예측 가능한 지역명 목록:
서울, 부산, 대구, 인천, 광주, 대전, 울산, 세종, 경기, 강원, 충북, 충남, 전북, 전남, 경북, 경남, 제주

예측 가능한 구분명 목록:
전체, 남, 여, 19-29, 30-39, 40-49, 50-59, 60-69, 70이상
예측할 지역명을 입력하세요: 서울
예측할 구분명을 쉼표로 구분하여 입력하세요: 19-29
서울 - 19-29 데이터 없음
[서울 - 19-29] 데이터가 없어 예측을 건너뜁니다.
